In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
import pdb

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True) 
from torch.utils.tensorboard import SummaryWriter


In [2]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)
import tensorboard
print(tensorboard.__version__)


1.1.0
0.3.0
1.15.0a20190806


In [3]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

0it [00:00, ?it/s]

100%|█████████▉| 26320896/26421880 [00:22<00:00, 1498107.30it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz



0it [00:00, ?it/s]


 56%|█████▌    | 16384/29515 [00:00<00:00, 80806.23it/s]
32768it [00:00, 33411.44it/s]                           
0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz



 99%|█████████▉| 4382720/4422102 [00:11<00:00, 1118048.13it/s]

0it [00:00, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz




  0%|          | 0/5148 [00:00<?, ?it/s]

8192it [00:00, 8878.35it/s]             

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [4]:
def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()


In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels =1,out_channels =6,kernel_size =5)
        self.conv2 = nn.Conv2d(in_channels =6,out_channels =12,kernel_size =5)
        
        self.fc1 = nn.Linear(in_features = 12*4*4, out_features = 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 60)
        self.out = nn.Linear(in_features= 60, out_features=10)
        
    def forward(self, t):
    # (1) input layer
        t = t

    # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

    # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

    # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

    # (6) output layer
        t = self.out(t)
    #t = F.softmax(t, dim=1)

        return t

In [7]:
batch_size_list = [100, 1000, 10000]
lr_list = [.01, .001, .0001, .00001]

In [8]:
for batch_size in batch_size_list:
    for lr in lr_list:
        network = Network()

        train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
        optimizer = optim.Adam(network.parameters(), lr=lr)

        images, labels = next(iter(train_loader))
        grid = torchvision.utils.make_grid(images)

        comment=f' batch_size={batch_size} lr={lr}'
        tb = SummaryWriter(comment=comment)
        tb.add_image('images', grid)
        tb.add_graph(network, images)

        for epoch in range(5):
            total_loss = 0
            total_correct = 0
            for batch in train_loader:
                images, labels = batch # Get Batch
                preds = network(images) # Pass Batch
                loss = F.cross_entropy(preds, labels) # Calculate Loss
                optimizer.zero_grad() # Zero Gradients
                loss.backward() # Calculate Gradients
                optimizer.step() # Update Weights

                total_loss += loss.item() * batch_size
                total_correct += get_num_correct(preds, labels)

            tb.add_scalar('Loss', total_loss, epoch)
            tb.add_scalar('Number Correct', total_correct, epoch)
            tb.add_scalar('Accuracy', total_correct / len(train_set), epoch)

            for name, param in network.named_parameters():
                tb.add_histogram(name, param, epoch)
                tb.add_histogram(f'{name}.grad', param.grad, epoch)

            print(
                "epoch", epoch
                ,"total_correct:", total_correct
                ,"loss:", total_loss
            )  
        tb.close()

epoch 0 total_correct: 46987 loss: 34041.04263037443
epoch 1 total_correct: 51378 loss: 23214.93953168392
epoch 2 total_correct: 52148 loss: 21314.07442688942
epoch 3 total_correct: 52505 loss: 20210.12680530548
epoch 4 total_correct: 52736 loss: 19449.157167971134
epoch 0 total_correct: 41987 loss: 47551.70609652996
epoch 1 total_correct: 48031 loss: 31881.6658526659
epoch 2 total_correct: 50041 loss: 27291.413055360317
epoch 3 total_correct: 51086 loss: 24515.468499064445
epoch 4 total_correct: 51664 loss: 22735.7948243618
epoch 0 total_correct: 32599 loss: 82135.21357774734
epoch 1 total_correct: 42017 loss: 48130.21501302719
epoch 2 total_correct: 44047 loss: 42532.71555900574
epoch 3 total_correct: 45199 loss: 39239.95760381222
epoch 4 total_correct: 46034 loss: 37074.31071102619
epoch 0 total_correct: 6656 loss: 137512.50383853912
epoch 1 total_correct: 25521 loss: 128613.3858203888
epoch 2 total_correct: 31065 loss: 102370.87032794952
epoch 3 total_correct: 37120 loss: 78893.592